In [ ]:
import pandas as pd
import os
import pickle
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
import easydict

In [2]:
def load_data(is_train=True):
    if is_train:
        sensor_path = './dataset/SWaT/SWaT_Dataset_Normal_v0.csv'
        df = pd.read_csv(sensor_path, index_col=0)
    else:
        sensor_path = './dataset/SWaT/SWaT_Dataset_Attack_v0.csv'
        df = pd.read_csv(sensor_path, index_col=0)

    for var_index in [item for item in df.columns if item != 'Normal/Attack']:
        df[var_index] = pd.to_numeric(df[var_index], errors='coerce')
    df.reset_index(drop=True, inplace=True)
    df.fillna(method='ffill', inplace=True)

    print(df.head())
    x = df.values[:,:-1].astype(np.float32)
    y = (df['Normal/Attack']=='Attack').to_numpy().astype(int)

    return x, y

In [3]:
train_x, train_y = load_data(True)
test_x, test_y = load_data(False)

     FIT101    LIT101  MV101  P101  P102    AIT201   AIT202   AIT203  \
0  2.470294  261.5804      2     2     1  244.3284  8.19008  306.101   
1  2.457163  261.1879      2     2     1  244.3284  8.19008  306.101   
2  2.439548  260.9131      2     2     1  244.3284  8.19008  306.101   
3  2.428338  260.2850      2     2     1  244.3284  8.19008  306.101   
4  2.424815  259.8925      2     2     1  244.4245  8.19008  306.101   

     FIT201  MV201  ...  P501  P502    PIT501  PIT502    PIT503    FIT601  \
0  2.471278      2  ...     1     1  10.02948     0.0  4.277749  0.000256   
1  2.468587      2  ...     1     1  10.02948     0.0  4.277749  0.000256   
2  2.467305      2  ...     1     1  10.02948     0.0  4.277749  0.000256   
3  2.466536      2  ...     1     1  10.02948     0.0  4.277749  0.000256   
4  2.466536      2  ...     1     1  10.02948     0.0  4.277749  0.000256   

   P601  P602  P603  Normal/Attack  
0     1     1     1         Normal  
1     1     1     1         No

In [4]:
x_min = np.min(train_x, 0, keepdims=True)
x_max = np.max(train_x, 0, keepdims=True)

In [5]:
args = easydict.EasyDict({
    "batch_size": 128, ## 배치 사이즈 설정
    "device": torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'), ## GPU 사용 여부 설정
    "input_size": train_x.shape[1], ## 입력 차원 설정
    "latent_size": 10, ## Hidden 차원 설정
    "output_size": train_x.shape[1], ## 출력 차원 설정
    "window_size" : 3, ## sequence Lenght
    "num_layers": 2,     ## LSTM layer 갯수 설정
    "learning_rate" : 0.001, ## learning rate 설정
    "max_iter" : 100000, ## 총 반복 횟수 설정
    'early_stop' : True,  ## valid loss가 작아지지 않으면 early stop 조건 설정
})

In [6]:
class SWaTDataset(Dataset):
    def __init__(self, x, y, x_min, x_max, window_size=1):
        super().__init__()
        t = (x_min != x_max).astype(np.float32)
        self.x = (x - x_min) / (x_max-x_min + 1e-5) * t
        self.y = y
        self.window_size = window_size

    def __len__(self):
        return self.x.shape[0] - self.window_size + 1

    def __getitem__(self, idx):
        return self.x[idx:idx+self.window_size], self.y[idx:idx+self.window_size]

In [8]:
train_dataset = SWaTDataset(train_x, train_y, x_min, x_max, window_size=args.window_size)
test_dataset = SWaTDataset(test_x, test_y, x_min, x_max, window_size=args.window_size)

train_loader = torch.utils.data.DataLoader(
                 dataset=train_dataset,
                 batch_size=args.batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_dataset,
                batch_size=args.batch_size,
                shuffle=False)

In [ ]:
from Omnianomaly_pytorch.model import OmniAnomaly

model = OmniAnomaly(
    in_dim=args.input_size,
    hidden_dim=10,
    z_dim=3,
    dense_dim=10,
    out_dim=args.input_size
)

model.to(args.device)

In [ ]:
def loss_function(x, pred_x, mu, logvar):
    MSE_loss = F.mse_loss(x, pred_x)
    KLD_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return MSE_loss + KLD_loss

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

In [ ]:
## Training
best_loss = None
epochs = tqdm(range(args.max_iter//len(train_loader)+1), leave=True)
for epoch in epochs:
    model.train()
    optimizer.zero_grad()
    train_iterator = tqdm(enumerate(train_loader), total=len(train_loader), desc="training", leave=True)

    train_loss = 0.0
    for i, batch_data in train_iterator:
        batch_data = batch_data[0].to(args.device)
        predict_values, mu, logvar = model(batch_data)

        # Backward and optimize

        optimizer.zero_grad()
        loss = loss_function(batch_data, predict_values, mu, logvar)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_iterator.set_postfix({
            "train_loss": float(loss),
        })

    train_loss = train_loss / len(train_loader)
    epochs.set_postfix({
         "Train Loss": train_loss,
    })

    model.eval()
    eval_loss = 0
    test_iterator = tqdm(enumerate(test_loader), total=len(test_loader), desc="testing", leave=True)
    with torch.no_grad():
        for i, batch_data in test_iterator:

            batch_data = batch_data[0].to(args.device)
            predict_values, mu, logvar = model(batch_data)
            loss = loss_function(batch_data, predict_values, mu, logvar)

            eval_loss += loss.mean().item()

            test_iterator.set_postfix({
                "eval_loss": float(loss),
            })

    eval_loss = eval_loss / len(test_loader)
    epochs.set_postfix({
         "Evaluation Score": float(eval_loss),
    })
    if best_loss is None or eval_loss < best_loss:
        best_loss = eval_loss
    else:
        if args.early_stop:
            print('early stop condition   best_loss[{}]  eval_loss[{}]'.format(best_loss, eval_loss))
            break


In [ ]:
def get_loss_list(args, model, test_loader):
    test_iterator = tqdm(enumerate(test_loader), total=len(test_loader), desc="testing")
    loss_list = []

    model.eval()
    with torch.no_grad():
        for i, batch_data in test_iterator:
            batch_data = batch_data[0].to(args.device)
            predict_values, _, _ = model(batch_data)

            loss = F.mse_loss(batch_data, predict_values, reduce=False)
            loss = loss.mean(dim=1).cpu().numpy()
            loss_list.append(loss)

    loss_list = np.concatenate(loss_list, axis=0)
    return loss_list

In [ ]:
test_loss_list = get_loss_list(args, model, test_loader)

In [ ]:
test_loss_list.shape

In [ ]:
anomaly_scores = np.mean(test_loss_list, axis = 1)

In [ ]:
plt.plot(anomaly_scores)

In [ ]:
x = list(range(len(anomaly_scores)))
y = anomaly_scores

window_y = []
for i in x:
    window_y.append(min(test_y[i:i+args.window_size])==0)

intervals = []
start = None
for i, label in enumerate(window_y):
    if label:
        if start is None:
            start = i
    else:
        if start is not None:
            intervals.append((start, i-1))
        start = None
if start is not None:
    intervals.append((start, len(window_y)-1))

In [ ]:
print(intervals)

In [ ]:
## 시각화 하기
fig = plt.figure(figsize=(16, 6))
ax=fig.add_subplot()

ax.plot(x, y)
plt.rcParams['axes.facecolor']='white'
for s, e in intervals:
    ax.axvspan(s, e, alpha=0.2, color='orange')

### Finding Threshold

In [ ]:
print(anomaly_scores)

In [ ]:
min, max = np.min(anomaly_scores), np.max(anomaly_scores)
min, max

In [ ]:
anomaly_scores.shape

In [ ]:
test_y.shape

In [ ]:
thresholds = np.linspace(min, max, 100)

In [ ]:
print(thresholds)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [ ]:
import seaborn as sns

In [ ]:
precisions = []
recalls = []
f1s = []

for threshold in thresholds:
    anomaly_prediction = (anomaly_scores > threshold).astype(int)
    target_y = test_y[args.window_size-1:]

    precisions.append(precision_score(target_y, anomaly_prediction, zero_division = 1))
    recalls.append(recall_score(target_y, anomaly_prediction, zero_division = 1))
    f1s.append(f1_score(target_y, anomaly_prediction, zero_division = 1))

In [ ]:
plt.plot(thresholds, precisions, label = "precision")
plt.plot(thresholds, recalls, label = "recall")
plt.plot(thresholds, f1s, label = "f1")
plt.legend()
plt.show()

In [ ]:
threshold_idx = np.argmax(f1s)

In [ ]:
best_threshold = thresholds[threshold_idx]

In [ ]:
fig = plt.figure(figsize=(16, 6))
ax=fig.add_subplot()
ax.axhline(y = best_threshold, color = "r")
ax.plot(x, y)
plt.rcParams['axes.facecolor']='white'
for s, e in intervals:
    ax.axvspan(s, e, alpha=0.2, color='orange')

In [ ]:
anomaly_prediction = (anomaly_scores > best_threshold).astype(int)
target_y = test_y[args.window_size-1:]

p, r, f = precision_score(target_y, anomaly_prediction), recall_score(target_y, anomaly_prediction),f1_score(target_y, anomaly_prediction)
p, r, f

In [ ]:
print(confusion_matrix(target_y, anomaly_prediction))

In [ ]:
sns.heatmap(confusion_matrix(target_y, anomaly_prediction), annot = True, fmt = "d")